In [34]:
import pandas as pd
original_data=pd.read_csv('D:\\student\\merged.csv')
original_cols=list(original_data.columns)
mcols=[]
cls=['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'nursery', 'internet', 'guardian.y', 'traveltime.y', 'studytime.y', 'failures.y', 'schoolsup.y', 'famsup.y', 'paid.y', 'activities.y', 'higher.y', 'romantic.y', 'famrel.y', 'freetime.y', 'goout.y', 'Dalc.y', 'Walc.y', 'health.y', 'absences.y', 'G1.y', 'G2.y', 'G3.y']
for col in original_cols[1:]:
    if col == 'guardian.y':
        break
    mcols.append(col)
pcols=[c for c in original_cols if c in cls]# portugese subject columns
maths_subject_data=original_data[mcols]
portugese_subject_data=original_data[pcols]

In [35]:
"""the following code in convert_categories is used for converting nominal category features like school,sex etc 
   to numeric categories"""

'the following code in convert_categories is used for converting nominal category features like school,sex etc \n   to numeric categories'

In [36]:
def convert_categories(datfr):#convert the nominal cateogries to numerical values
    dcols=list(datfr.columns)
    for col in dcols:
        if datfr[col].dtype=='object':#if the column has nominal categorical data
            datfr[col]=datfr[col].astype('category')
            datfr[col]=datfr[col].cat.codes
    return datfr
maths_processed=convert_categories(maths_subject_data)
portugese_processed=convert_categories(portugese_subject_data)
existing_rows=maths_processed.shape[0]#number of students in original dataset

C:\Users\sudhe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\sudhe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
"""the code in the following cell takes a set of features and then generate a bigger feature dataset based on distribution 
of existing values in each column.
 calculate_percentages(column)-> calculates the percentages of different unique values in the column
get_extra_data()-> based on percentages calculated for different values in a column they are added randomly to form 
a bigger dataset"""

'the code in the following cell takes a set of features and then generate a bigger feature dataset based on distribution \nof existing values in each column.\n calculate_percentages(column)-> calculates the percentages of different unique values in the column\nget_extra_data()-> based on percentages calculated for different values in a column they are added randomly to form \na bigger dataset'

In [38]:
from collections import Counter
import random
#random data generation based on existing distribution
def calculate_percentages(column):#cl-> column
    column_data=list(column)#create a list
    counts_pattern=Counter(column_data)
    total_count=len(column_data)
    percentages={}
    for element,count in counts_pattern.items():
        if element not in percentages.keys():
            percentages[element]=round(count/total_count,3)#calculate the percentage of no of times that particular data item has appeared
    return percentages
def get_extra_data(per,number):#create synthetic data based on existing distribution
    possible={}
    for item,p in per.items():
        if item not in possible.keys():
            occur=round(p*number,0)
            possible[item]=int(occur)
    total_count=sum(possible.values())
    high=max(possible.values())
    low=min(possible.values())
    print(total_count)
    if total_count>number:#if the total count exceeds the possibility then subtract the diff from highest occurring element
        #print('sudheer')
        diff=total_count-number
        for element,count in possible.items():
            if count==high:
                possible[element]=count-diff
                break
    if total_count<number:#if the total count is short of possibility then add the diff to the one of the lowest occurring element
        #print('kumar')
        diff=number-total_count
        for element,count in possible.items():
            if count==low:
                possible[element]=count+diff
                break
                
    synthetic_data=[]
    for item,occur in possible.items():
        for i in range(occur):
            synthetic_data.append(item)
    print(len(synthetic_data))
    jumbled_synthetic_data=[]
    tracking_data=[]
    synthetic_indices=list(range(len(synthetic_data)))
    while True:#jumbling the synthetic data list
        a=random.choice(synthetic_indices)
        if a not in tracking_data:
            data=synthetic_data[a]
            jumbled_synthetic_data.append(data)
        if len(jumbled_synthetic_data)==len(synthetic_data):
            break
    return jumbled_synthetic_data

In [39]:
"""generate synthetic data function calls the calculate percentages and get extra data functions 
and gathers a list of list consisting of original data and synthetic data .Then a dataframe is created for 
the bigger dataset"""

'generate synthetic data function calls the calculate percentages and get extra data functions \nand gathers a list of list consisting of original data and synthetic data .Then a dataframe is created for \nthe bigger dataset'

In [40]:
#generate synthetic data for all columns and store that data in a dataframe
def generate_synthetic_data(additional,df,cols):
    final_data=[]
    expanded_data=[]
    for i in cols:#only features
        dt=df[i]
        actual=list(dt)
        print('...............................')
        percents=calculate_percentages(dt)
        extra=get_extra_data(percents,additional)
        actual.extend(extra)
        final_data.append(actual)
        print('**********************************************') 
    expanded_length=existing_rows+additional
    for i in range(expanded_length):#re-arranging list of lists to create dataframe
        t=[]
        for j in range(len(final_data)):
            t.append(final_data[j][i])
        expanded_data.append(t)
    print(len(expanded_data))
    bigger_data=pd.DataFrame(expanded_data,columns=cols)
    return bigger_data



In [41]:
""" the following code fits linear regression over original feature data and response variables and then it would predict
the marks for synthetic data thereby it reduces the randomness in the synthetic data and establishes some 
relationship  which would make the additional synthetic data valuable in improving the fit of the model
This procedure is implemented for both maths and portugese subjects separately .Later the two dataframes are joinned together
The synthetic data for demographic features is generated only once since these features are common in both subjects"""

' the following code fits linear regression over original feature data and response variables and then it would predict\nthe marks for synthetic data thereby it reduces the randomness in the synthetic data and establishes some \nrelationship  which would make the additional synthetic data valuable in improving the fit of the model\nThis procedure is implemented for both maths and portugese subjects separately .Later the two dataframes are joinned together\nThe synthetic data for demographic features is generated only once since these features are common in both subjects'

In [42]:
more_students=int(input('please enter the additional number of students you would like to addd to the exisiting dataset: '))
labels=['G1.x','G2.x','G3.x']#maths subject marks columns
port_labels=['G1.y','G2.y','G3.y']#portugese subject marks columns
feature_cols=mcols[:-3]#excludes all three marks columns i.e only features of maths subject
port_feature_cols=pcols[:-3]#feature columns for portugese subject
feature_data=maths_processed[feature_cols]#maths subject feature data
port_feature_data=portugese_processed[port_feature_cols]#portugese subject features
big_data=generate_synthetic_data(more_students,feature_data,feature_cols)#get bigger dataset for maths subject 
#extract the demographic data from maths bigger dataset since they are common to both
pdemo_data=big_data[['school', 'sex', 'age', 'address', 'famsize', 'Pstatus','Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'nursery', 'internet']]#demographic data for portugese subject
#generate synthetic data for portugese subject
pdata=generate_synthetic_data(more_students,port_feature_data,port_feature_cols)#portugese features synthetic data
#filtering only school-related features data for portugese subject
pschool_data=pdata[['guardian.y','traveltime.y', 'studytime.y', 'failures.y', 'schoolsup.y', 'famsup.y','paid.y', 'activities.y', 'higher.y', 'romantic.y', 'famrel.y','freetime.y', 'goout.y', 'Dalc.y', 'Walc.y', 'health.y', 'absences.y']]
portugese_big_data=pdemo_data.join(pschool_data)#bigger dataset for portugese subject
#creating a dataframe
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
for lab in labels:
    label_data=maths_processed[lab]
    #lnr=LinearRegression()
    #lnr=DecisionTreeRegressor()
    lnr=RandomForestRegressor(max_depth=2, random_state=0)
    lnr.fit(feature_data,label_data)#fitting a linear regression model on small dataset
    necc_data=big_data[382:]#synthetic data
    predicted=lnr.predict(necc_data)#predicting only on synthetic data
    proper_predicted=[int(round(i,0)) for i in predicted]#convert float to int along with round off numbers 
    orig_marks=list(label_data)
    feature_data[lab]=orig_marks
    orig_marks.extend(proper_predicted)
    big_data[lab]=orig_marks
    print('generated %r marks based on synthetic data' % lab)
for lab in port_labels:
    label_data=portugese_processed[lab]
    lnr=LinearRegression()
    lnr.fit(port_feature_data,label_data)#fitting a linear regression model on small dataset
    necc_data=portugese_big_data[382:]#synthetic data
    predicted=lnr.predict(necc_data)#predicting only on synthetic data
    proper_predicted=[int(round(i,0)) for i in predicted]#convert float to int along with round off numbers 
    orig_marks=list(label_data)
    port_feature_data[lab]=orig_marks
    orig_marks.extend(proper_predicted)
    portugese_big_data[lab]=orig_marks
    print('generated %r marks of portugese subject based on synthetic data' % lab)
    
print(big_data.shape,portugese_big_data.shape)

please enter the additional number of students you would like to addd to the exisiting dataset: 500
...............................
500
500
**********************************************
...............................
500
500
**********************************************
...............................
501
500
**********************************************
...............................
500
500
**********************************************
...............................
500
500
**********************************************
...............................
500
500
**********************************************
...............................
499
500
**********************************************
...............................
501
500
**********************************************
...............................
500
500
**********************************************
...............................
499
500
**********************************************
..............................

C:\Users\sudhe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\sudhe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\sudhe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [43]:
#combine both maths and portugese dataframes and saved it to the csv file to form combined bigger dataset

In [44]:
portu_unique=['guardian.y','traveltime.y', 'studytime.y', 'failures.y', 'schoolsup.y', 'famsup.y','paid.y', 'activities.y', 'higher.y', 'romantic.y', 'famrel.y','freetime.y', 'goout.y', 'Dalc.y', 'Walc.y', 'health.y', 'absences.y',
       'G1.y', 'G2.y', 'G3.y']
comp_data=big_data.join(portugese_big_data[portu_unique])
comp_data.to_csv('D:\\student\\final_combined_daatset.csv',index=False)